<a href="https://colab.research.google.com/github/Gokcentunc/atraction/blob/main/Atraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4leaves 4 states

In [ ]:
import sympy as sp
from itertools import product, permutations
from functools import reduce

# ============================================================
# OUTPUT
# ============================================================
OUT_MAIN_TEX = "p0_q_tildeq_4leaves_4states.tex"

# ============================================================
# SYMBOLS
# ============================================================
pi1, pi2, pi3, pi4 = sp.symbols('pi1 pi2 pi3 pi4', nonzero=True)
pi = [pi1, pi2, pi3, pi4]

lambda_1 = sp.Symbol('lambda_1')  # prints \lambda_{1}
lam = sp.Symbol('lambda')         # prints \lambda
lam_vec = [lambda_1, lam, lam, lam]

pi12  = pi1 + pi2
pi123 = pi1 + pi2 + pi3
pisum = pi1 + pi2 + pi3 + pi4

# ============================================================
# BASIS VECTORS u^1,...,u^4
# ============================================================
u1 = sp.Matrix([pi1, pi2, pi3, pi4])          # u^1 = pi
u2 = sp.Matrix([pi1, pi2, pi3, -pi123])       # u^2
u3 = sp.Matrix([pi1, pi2, -pi12, 0])          # u^3
u4 = sp.Matrix([pi1, -pi1, 0, 0])             # u^4
u = [u1, u2, u3, u4]

def ip_pi(x, y):
    return sum((x[j] * y[j]) / pi[j] for j in range(4))

def e_u(j, ui):
    return ui[j] / pi[j]

norm = [sp.simplify(ip_pi(ui, ui)) for ui in u]

def simp(expr):
    expr = sp.together(sp.simplify(expr))
    expr = expr.subs({pisum: 1})
    expr = expr.subs({pi123: 1 - pi4})
    expr = expr.subs({pi1 + pi2: pi12})
    return sp.simplify(sp.together(expr))

# ============================================================
# p^0
# ============================================================
def p0_n_raw(idx):
    denom = reduce(lambda a, b: a * b, (norm[i - 1] for i in idx), 1)
    s = 0
    for j in range(4):
        prod_term = reduce(lambda a, b: a * b, (e_u(j, u[i - 1]) for i in idx), 1)
        s += pi[j] * prod_term
    return sp.simplify(s / denom)

def p0_4leaves():
    p0 = {}
    for idx in product(range(1, 5), repeat=4):
        p0[idx] = simp(p0_n_raw(idx))
    return p0

def p0_tripod():
    p0T = {}
    for a, b, c in product(range(1, 5), repeat=3):
        p0T[(a, b, c)] = simp(p0_n_raw((a, b, c)))
    return p0T

# ============================================================
# bar q via gluing for quartet topology
#   12|34: (i1,i2,y) + (y,i3,i4)
#   13|24: (i1,i3,y) + (y,i2,i4)
#   14|23: (i1,i4,y) + (y,i2,i3)
# ============================================================
def q_4leaves(topology="12|34"):
    p0T = p0_tripod()
    q = {}

    def left_right_triples(i1, i2, i3, i4, y):
        if topology == "12|34":
            return (i1, i2, y), (y, i3, i4)
        if topology == "13|24":
            return (i1, i3, y), (y, i2, i4)
        if topology == "14|23":
            return (i1, i4, y), (y, i2, i3)
        raise ValueError("Unknown topology. Use '12|34', '13|24', or '14|23'.")

    for i1, i2, i3, i4 in product(range(1, 5), repeat=4):
        s = 0
        for y in range(1, 5):
            L, R = left_right_triples(i1, i2, i3, i4, y)
            s += norm[y - 1] * (lam_vec[y - 1] * p0T[L]) * p0T[R]
        q[(i1, i2, i3, i4)] = simp(s)

    return q

# ============================================================
# tilde q = bar q / p^0
# ============================================================
def tilde_q(q, p0_4):
    tq = {}
    for idx, val in q.items():
        den = sp.simplify(p0_4[idx])
        if den != 0:
            tq[idx] = simp(sp.simplify(val / den))
        else:
            tq[idx] = simp(sp.simplify(val))
    return tq

# ============================================================
# Symmetry grouping helpers
# ============================================================
def orbit_S4(idx):
    return sorted(set(permutations(idx, 4)))

def canon_S4(idx):
    return tuple(sorted(idx))

def orbit_swap_positions(idx, swaps):
    res = set()
    base = list(idx)
    k = len(swaps)
    for mask in range(2 ** k):
        v = base[:]
        for t in range(k):
            if (mask >> t) & 1:
                a, b = swaps[t]
                v[a], v[b] = v[b], v[a]
        res.add(tuple(v))
    return sorted(res)

def orbit_topology(idx, topology):
    if topology == "12|34":
        return orbit_swap_positions(idx, [(0, 1), (2, 3)])
    if topology == "13|24":
        return orbit_swap_positions(idx, [(0, 2), (1, 3)])
    if topology == "14|23":
        return orbit_swap_positions(idx, [(0, 3), (1, 2)])
    raise ValueError("Unknown topology")

# ============================================================
# TeX PREAMBLE / ENDDOC (your exact snippet)
# ============================================================
PREAMBLE = r"""\documentclass[12pt,a4paper]{amsart}
\usepackage{amsfonts}
\usepackage{amsthm}
\usepackage{amsmath}
\usepackage{amssymb}
\usepackage[margin=2cm]{geometry}
\usepackage[utf8]{inputenc}
\usepackage[T1]{fontenc}
\usepackage{enumitem}
\begin{document}
"""
ENDDOC = r"\end{document}"

# ============================================================
# TeX writer: full document now
# ============================================================
def write_all_tex_multi(p0, q_dict, tq_dict, filename=OUT_MAIN_TEX, skip_zeros=True):
    lines = [PREAMBLE]

    # ---- p0 (once, S4) ----
    lines.append(r"\section*{No-evolution point $\bar{p}^0$ for $n=4$ leaves, $\kappa=4$ states}")
    lines.append(r"\begin{enumerate}[label=\textbf{\small (\arabic*)}]")
    reps = sorted({canon_S4(idx) for idx in p0.keys()})
    for rep0 in reps:
        O = orbit_S4(rep0)
        val = sp.simplify(p0[rep0])
        if skip_zeros and val == 0:
            continue
        chain = " = ".join([rf"\bar{{p}}^0_{{{a}{b}{c}{d}}}" for (a, b, c, d) in O])
        lines.append(rf"\item ${chain} = {sp.latex(val)}$")
    lines.append(r"\end{enumerate}")
    lines.append("")

    # ---- For each topology: q and tilde q grouped by the right symmetry ----
    for topo in ["12|34", "13|24", "14|23"]:
        q = q_dict[topo]
        tq = tq_dict[topo]

        lines.append(r"\section*{$\bar{q}$ for " + topo + r" with $\lambda_2=\lambda_3=\lambda_4=\lambda$}")
        lines.append(r"\begin{enumerate}[label=\textbf{\small (\arabic*)}]")
        visited = set()
        for idx in sorted(q.keys()):
            if idx in visited:
                continue
            O = orbit_topology(idx, topo)
            visited.update(O)
            val = sp.simplify(q[idx])
            if skip_zeros and val == 0:
                continue
            chain = " = ".join([rf"\bar{{q}}_{{{a}{b}{c}{d}}}" for (a, b, c, d) in O])
            lines.append(rf"\item ${chain} = {sp.latex(val)}$")
        lines.append(r"\end{enumerate}")
        lines.append("")

        lines.append(r"\section*{$\tilde{q}=\bar{q}/\bar{p}^0$ for " + topo + r" (grouped by the topology symmetry)}")
        lines.append(r"\begin{enumerate}[label=\textbf{\small (\arabic*)}]")
        visited = set()
        for idx in sorted(tq.keys()):
            if idx in visited:
                continue
            O = orbit_topology(idx, topo)
            visited.update(O)
            val = sp.simplify(tq[idx])
            if skip_zeros and val == 0:
                continue
            chain = " = ".join([rf"\tilde{{q}}_{{{a}{b}{c}{d}}}" for (a, b, c, d) in O])
            lines.append(rf"\item ${chain} = {sp.latex(val)}$")
        lines.append(r"\end{enumerate}")
        lines.append("")

    lines.append(ENDDOC)

    with open(filename, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))
    print(f"Wrote TeX file: {filename}")

# ============================================================
# MAIN
# ============================================================
def main():
    p0 = p0_4leaves()

    q_dict = {}
    tq_dict = {}
    for topo in ["12|34", "13|24", "14|23"]:
        q_dict[topo] = q_4leaves(topology=topo)
        tq_dict[topo] = tilde_q(q_dict[topo], p0)

    write_all_tex_multi(p0, q_dict, tq_dict, filename=OUT_MAIN_TEX, skip_zeros=True)

    # optional sanity prints
    for topo in ["12|34", "13|24", "14|23"]:
        print(f"\nSanity checks ({topo}):")
        print("  p0_1111 =", sp.latex(p0[(1, 1, 1, 1)]))
        print("  q_1111  =", sp.latex(q_dict[topo][(1, 1, 1, 1)]))
        print("  tq_1111 =", sp.latex(tq_dict[topo][(1, 1, 1, 1)]))

if __name__ == "__main__":
    main()

# 5leaves 4 states

In [ ]:
import sympy as sp
from itertools import product
from functools import reduce

# ============================================================
# OUTPUT
# ============================================================
OUT_MAIN_TEX = "tildeq_5leaves_4states_grouped.tex"

# ============================================================
# SWITCHES
# ============================================================
SKIP_ZEROS = True
EXCLUDE_STATE_1 = True   # 1 içeren tüm indeksler atılacak

# TeX formatting controls
MAX_LHS_PER_LINE = 10
USE_SMALL_FOR_LONG = True
SMALL_LHS_THRESHOLD = 18

# Split one big fraction into multiple fractions with same denominator
SPLIT_LONG_FRACTIONS = True
FRAC_TERMS_PER_LINE = 8
FRAC_MAX_FRACTIONS = 10   # biraz yükselttim (istersen azalt)

# ============================================================
# SYMBOLS
# ============================================================
pi1, pi2, pi3, pi4 = sp.symbols("pi1 pi2 pi3 pi4", nonzero=True)
pi = [pi1, pi2, pi3, pi4]

pi12  = pi1 + pi2
pi123 = pi1 + pi2 + pi3
pisum = pi1 + pi2 + pi3 + pi4

# edge 6 (quartet)
lam1_6 = sp.Symbol("lambda_1^6")
lam6   = sp.Symbol("lambda^6")
lam_vec6 = [lam1_6, lam6, lam6, lam6]

# edge 7 (gluing)
lam1_7 = sp.Symbol("lambda_1^7")
lam7   = sp.Symbol("lambda^7")
lam_vec7 = [lam1_7, lam7, lam7, lam7]

# ============================================================
# BASIS VECTORS
# ============================================================
u1 = sp.Matrix([pi1, pi2, pi3, pi4])
u2 = sp.Matrix([pi1, pi2, pi3, -pi123])
u3 = sp.Matrix([pi1, pi2, -pi12, 0])
u4 = sp.Matrix([pi1, -pi1, 0, 0])
u = [u1, u2, u3, u4]

def ip_pi(x, y):
    return sum((x[j] * y[j]) / pi[j] for j in range(4))

def e_u(j, ui):
    return ui[j] / pi[j]

# norm hesapları az sayıda -> burada simplify sorun değil
norm = [sp.simplify(ip_pi(ui, ui)) for ui in u]

# ============================================================
# VERY LIGHT simplification: ONLY simplex relation
# ============================================================
SUBS_RULES = {pisum: 1, pi123: 1 - pi4, (pi1 + pi2): pi12}

def simp_light(expr):
    # Ağır simplify yok: sadece subs + together + cancel
    expr = expr.subs(SUBS_RULES)
    expr = sp.together(expr)
    expr = sp.cancel(expr)
    return expr

# ============================================================
# Filters
# ============================================================
def keep_idx(idx):
    return not (EXCLUDE_STATE_1 and (1 in idx))

def is_nan_like(expr):
    return (expr is sp.nan) or (expr == sp.nan) or (hasattr(expr, "has") and expr.has(sp.nan))

# ============================================================
# p^0 (needed for tilde q, NOT printed)
# ============================================================
def p0_n_raw(idx):
    denom = reduce(lambda a, b: a * b, (norm[i - 1] for i in idx), 1)
    s = 0
    for j in range(4):
        prod_term = reduce(lambda a, b: a * b, (e_u(j, u[i - 1]) for i in idx), 1)
        s += pi[j] * prod_term
    return s / denom

def p0_tripod_full():
    p0T = {}
    for idx in product([1,2,3,4], repeat=3):
        p0T[idx] = simp_light(p0_n_raw(idx))
    return p0T

def p0_5_only(states_out):
    p0 = {}
    for idx in product(states_out, repeat=5):
        p0[idx] = simp_light(p0_n_raw(idx))
    return p0

# ============================================================
# bar q computations (internal only)
# ============================================================
def barq_quartet_edge6_full(p0T):
    q4 = {}
    for i1,i2,i3,is_ in product([1,2,3,4], repeat=4):
        s = 0
        for y in [1,2,3,4]:
            s += norm[y-1] * lam_vec6[y-1] * p0T[(i1,i2,y)] * p0T[(y,i3,is_)]
        q4[(i1,i2,i3,is_)] = simp_light(s)
    return q4

def barq_5leaves_only(states_out, barq_T1, p0T):
    q5 = {}
    for i1,i2,i3,i4,i5 in product(states_out, repeat=5):
        s = 0
        for x in [1,2,3,4]:
            s += norm[x-1] * lam_vec7[x-1] * barq_T1[(i1,i2,i3,x)] * p0T[(x,i4,i5)]
        q5[(i1,i2,i3,i4,i5)] = simp_light(s)
    return q5

# ============================================================
# tilde q = barq / p0  (LIGHT)
# ============================================================
def tilde_q_only(barq5, p05):
    tq = {}
    for idx, num in barq5.items():
        den = p05[idx]
        if den == 0:
            tq[idx] = sp.nan
        else:
            tq[idx] = simp_light(num / den)
    return tq

# ============================================================
# Group by exact same expression (after light simp)
# ============================================================
def group_equal_expressions(expr_dict):
    groups = {}
    for idx, val in expr_dict.items():
        if not keep_idx(idx):
            continue
        if is_nan_like(val):
            continue
        val = simp_light(val)
        if SKIP_ZEROS and val == 0:
            continue
        key = sp.srepr(val)  # exact structural key
        groups.setdefault(key, {"val": val, "idxs": []})
        groups[key]["idxs"].append(idx)

    out = []
    for key in sorted(groups.keys(), key=lambda k: groups[k]["idxs"][0]):
        out.append((sorted(groups[key]["idxs"]), groups[key]["val"]))
    return out

# ============================================================
# TeX formatting helpers
# ============================================================
def tex_idx(idx):
    return "".join(str(i) for i in idx)

def lhs_substack(symbol_tex, idxs, chunk_size=MAX_LHS_PER_LINE):
    items = [rf"{symbol_tex}_{{{tex_idx(idx)}}}" for idx in idxs]
    chunks = [items[i:i+chunk_size] for i in range(0, len(items), chunk_size)]
    lines = [" = ".join(ch) for ch in chunks]
    if len(lines) == 1:
        return lines[0], len(items)
    return r"\substack{" + r"\\ ".join(lines) + r"}", len(items)

def split_fraction_to_tex_lines(expr, terms_per_frac=FRAC_TERMS_PER_LINE, max_fracs=FRAC_MAX_FRACTIONS):
    expr = simp_light(expr)
    expr = sp.together(expr)
    num, den = sp.fraction(expr)

    if den == 1 or den == -1:
        return None

    # num'u expand ETME -> çok yavaş. Sadece Add ise terimlere ayır.
    terms = sp.Add.make_args(num) if isinstance(num, sp.Add) else (num,)
    chunks = [terms[i:i+terms_per_frac] for i in range(0, len(terms), terms_per_frac)]
    if len(chunks) <= 1 or len(chunks) > max_fracs:
        return None

    den_tex = sp.latex(den)

    lines = []
    for k, ch in enumerate(chunks):
        chunk_expr = sp.Add(*ch)
        chunk_tex = sp.latex(chunk_expr)
        frac_tex = rf"\frac{{{chunk_tex}}}{{{den_tex}}}"

        if k == 0:
            lines.append(frac_tex)
        else:
            # işaret chunk içinde kalabilir; aligned satırında taşma açısından sorun değil
            lines.append(r"+ " + frac_tex)
    return lines

def rhs_lines(expr):
    expr = simp_light(expr)
    if SPLIT_LONG_FRACTIONS:
        frac_lines = split_fraction_to_tex_lines(expr)
        if frac_lines is not None:
            return frac_lines
    # fallback: tek satır
    return [sp.latex(expr)]

def make_aligned_block(symbol_tex, idxs, rhs_expr):
    lhs, n_items = lhs_substack(symbol_tex, idxs)
    rhs_lines_list = rhs_lines(rhs_expr)

    use_small = USE_SMALL_FOR_LONG and (n_items >= SMALL_LHS_THRESHOLD)

    rows = [rf"{lhs} &= {rhs_lines_list[0]}"]
    for j in range(1, len(rhs_lines_list)):
        rows.append(rf"&\quad {rhs_lines_list[j]}")

    body = " \\\\\n".join(rows)

    if use_small:
        return (
            r"\[" "\n"
            r"\small" "\n"
            r"\begin{aligned}" "\n"
            + body + "\n"
            r"\end{aligned}" "\n"
            r"\normalsize" "\n"
            r"\]"
        )
    else:
        return (
            r"\[" "\n"
            r"\begin{aligned}" "\n"
            + body + "\n"
            r"\end{aligned}" "\n"
            r"\]"
        )

# ============================================================
# TeX writer (ONLY tilde q)
# ============================================================
PREAMBLE = r"""\documentclass[12pt]{amsart}
\usepackage{amsmath,amssymb}
\usepackage[margin=1.6cm]{geometry}
\allowdisplaybreaks
\begin{document}
"""
ENDDOC = r"\end{document}"

def write_tex_only_tildeq(tildeq_5, filename):
    lines = [PREAMBLE]
    lines.append(r"\section*{$\tilde{q}$ for $5$ leaves}")

    for idxs, val in group_equal_expressions(tildeq_5):
        lines.append(make_aligned_block(r"\tilde{q}", idxs, val))
        lines.append("")

    lines.append(ENDDOC)

    with open(filename, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))

    print("TeX written to:", filename)

# ============================================================
# MAIN
# ============================================================
def main():
    states_out = [2,3,4] if EXCLUDE_STATE_1 else [1,2,3,4]

    # tripod p0 once
    p0T = p0_tripod_full()

    # quartet barq once
    barq_T1 = barq_quartet_edge6_full(p0T)

    # only needed 5-leaf indices: 3^5=243 (if exclude 1)
    p0_5 = p0_5_only(states_out)
    barq_5 = barq_5leaves_only(states_out, barq_T1, p0T)

    # tilde q
    tildeq_5 = tilde_q_only(barq_5, p0_5)

    # write only tilde q
    write_tex_only_tildeq(tildeq_5, OUT_MAIN_TEX)

if __name__ == "__main__":
    main()


# Equalities 4-states

In [ ]:
import sympy as sp
from functools import reduce, lru_cache

# ============================================================
# SETTINGS
# ============================================================
EXCLUDE_STATE_1 = True
LIGHT_SIMPLIFY = True

# ============================================================
# SYMBOLS (4-states fixed)
# ============================================================
pi1, pi2, pi3, pi4 = sp.symbols("pi1 pi2 pi3 pi4", nonzero=True)
pi = [pi1, pi2, pi3, pi4]

pi12  = pi1 + pi2
pi123 = pi1 + pi2 + pi3
pisum = pi1 + pi2 + pi3 + pi4

lam1_6 = sp.Symbol("lambda_1^6")
lam6   = sp.Symbol("lambda^6")
lam_vec6 = [lam1_6, lam6, lam6, lam6]

lam1_7 = sp.Symbol("lambda_1^7")
lam7   = sp.Symbol("lambda^7")
lam_vec7 = [lam1_7, lam7, lam7, lam7]

lam1_8 = sp.Symbol("lambda_1^8")
lam8   = sp.Symbol("lambda^8")
lam_vec8 = [lam1_8, lam8, lam8, lam8]

# ============================================================
# BASIS (your working choice)
# ============================================================
u1 = sp.Matrix([pi1, pi2, pi3, pi4])
u2 = sp.Matrix([pi1, pi2, pi3, -pi123])
u3 = sp.Matrix([pi1, pi2, -pi12, 0])
u4 = sp.Matrix([pi1, -pi1, 0, 0])
u = [u1, u2, u3, u4]

def ip_pi(x, y):
    return sum((x[j] * y[j]) / pi[j] for j in range(4))

def e_u(j, ui):
    return ui[j] / pi[j]

norm = [sp.simplify(ip_pi(ui, ui)) for ui in u]

# ============================================================
# LIGHT SIMPLIFICATION (your intended rules)
# ============================================================
SUBS_RULES = {
    pisum: 1,
    pi123: 1 - pi4,
    (pi1 + pi2): pi12,
}

def simp_light(expr):
    if not LIGHT_SIMPLIFY:
        return expr
    expr = expr.subs(SUBS_RULES)
    expr = sp.together(expr)
    expr = sp.cancel(expr)
    expr = expr.subs(SUBS_RULES)
    return expr

# ============================================================
# Index helpers
# ============================================================
def parse_idx(s: str):
    s = s.strip().replace(" ", "")
    if not s or not all(ch.isdigit() for ch in s):
        raise ValueError("Index must be digits like 2244, 22444, 425544.")
    return tuple(int(ch) for ch in s)

def keep_idx(idx):
    return not (EXCLUDE_STATE_1 and (1 in idx))

# ============================================================
# p0_bar (cached)
# ============================================================
@lru_cache(maxsize=None)
def p0_bar(idx):
    denom = reduce(lambda a, b: a * b, (norm[i - 1] for i in idx), 1)
    s = 0
    for j in range(4):
        prod_term = reduce(lambda a, b: a * b, (e_u(j, u[i - 1]) for i in idx), 1)
        s += pi[j] * prod_term
    return simp_light(s / denom)

# ============================================================
# barq / tildeq
# ============================================================
@lru_cache(maxsize=None)
def barq4(idx4):
    i1, i2, i3, i4 = idx4
    s = 0
    for y in [1, 2, 3, 4]:
        s += norm[y-1] * lam_vec6[y-1] * p0_bar((i1, i2, y)) * p0_bar((y, i3, i4))
    return simp_light(s)

@lru_cache(maxsize=None)
def tildeq4(idx4):
    den = p0_bar(idx4)
    if den == 0:
        return sp.nan
    return simp_light(barq4(idx4) / den)

@lru_cache(maxsize=None)
def barq5(idx5):
    i1, i2, i3, i4, i5 = idx5
    s = 0
    for x in [1, 2, 3, 4]:
        s += norm[x-1] * lam_vec7[x-1] * barq4((i1, i2, i3, x)) * p0_bar((x, i4, i5))
    return simp_light(s)

@lru_cache(maxsize=None)
def tildeq5(idx5):
    den = p0_bar(idx5)
    if den == 0:
        return sp.nan
    return simp_light(barq5(idx5) / den)

@lru_cache(maxsize=None)
def barq6(idx6):
    i1, i2, i3, i4, i5, i6 = idx6
    s = 0
    for x in [1, 2, 3, 4]:
        s += norm[x-1] * lam_vec8[x-1] * barq5((i1, i2, i3, i4, x)) * p0_bar((x, i5, i6))
    return simp_light(s)

@lru_cache(maxsize=None)
def tildeq6(idx6):
    den = p0_bar(idx6)
    if den == 0:
        return sp.nan
    return simp_light(barq6(idx6) / den)

# ============================================================
# Equality under simplex (FIXED Groebner domain)
# ============================================================
def equal_under_simplex(expr):
    """
    Decide expr == 0 under pisum=1 by reducing the NUMERATOR modulo <pisum-1>.
    IMPORTANT: use domain='EX' so coefficients may involve lambdas.
    """
    expr = simp_light(expr)
    expr = sp.together(expr)
    num, den = sp.fraction(expr)

    num = sp.expand(num)

    # Groebner over pi-variables, coefficients in expression domain (allows lambdas)
    G = sp.groebner([pisum - 1], pi1, pi2, pi3, pi4, order="lex", domain="EX")

    rem = G.reduce(num)[1]
    rem = sp.factor(rem)
    return (rem == 0), rem, den

# ============================================================
# Interactive compare
# ============================================================
def main():
    mode = input("Choose object (quartet / 5 / 6): ").strip().lower()
    if mode not in ["quartet", "5", "6"]:
        raise ValueError("Mode must be one of: quartet, 5, 6.")

    s1 = input("Enter index #1 (e.g. 2244 / 22444 / 425544): ").strip()
    s2 = input("Enter index #2: ").strip()

    idx1 = parse_idx(s1)
    idx2 = parse_idx(s2)

    if not keep_idx(idx1) or not keep_idx(idx2):
        raise ValueError("Index rejected: it contains state 1 while EXCLUDE_STATE_1=True.")

    if mode == "quartet":
        if len(idx1) != 4 or len(idx2) != 4:
            raise ValueError("quartet requires length-4 indices.")
        v1 = tildeq4(idx1)
        v2 = tildeq4(idx2)

    elif mode == "5":
        if len(idx1) != 5 or len(idx2) != 5:
            raise ValueError("5 requires length-5 indices.")
        v1 = tildeq5(idx1)
        v2 = tildeq5(idx2)

    else:  # mode == "6"
        if len(idx1) != 6 or len(idx2) != 6:
            raise ValueError("6 requires length-6 indices.")
        v1 = tildeq6(idx1)
        v2 = tildeq6(idx2)

    diff = v1 - v2

    naive_is_zero = (simp_light(diff) == 0)
    is_zero, rem, den = equal_under_simplex(diff)

    print("\n--- Results ---")
    print(f"tilde({s1}) =\n{v1}")
    print(f"\ntilde({s2}) =\n{v2}")

    print("\n--- Equality check ---")
    print("Naive (diff==0 after light simp) ->", naive_is_zero)
    print("Under constraint pi1+pi2+pi3+pi4=1 ->", is_zero)

    if not is_zero:
        print("\nRemainder of numerator mod <pisum-1>:")
        print(rem)
        print("\nDenominator (for reference):")
        print(den)

if __name__ == "__main__":
    main()


Choose object (quartet / 5 / 6): 5
Enter index #1 (e.g. 2244 / 22444 / 425544): 24444
Enter index #2: 34444

--- Results ---
tilde(24444) =
(-lambda^6*lambda^7*pi1**2*pi2*pi4 + lambda^6*lambda^7*pi1**2*pi4 - lambda^6*lambda^7*pi1**2 - lambda^6*lambda^7*pi1*pi2**2*pi4 - lambda^6*lambda^7*pi1*pi2*pi3 - 2*lambda^6*lambda^7*pi1*pi2*pi4 + 2*lambda^6*lambda^7*pi1*pi2 + lambda^6*lambda^7*pi2**2*pi4 - lambda^6*lambda^7*pi2**2 + lambda^6*lambda_1^7*pi1**2*pi2*pi4 - lambda^6*lambda_1^7*pi1**2*pi2 + lambda^6*lambda_1^7*pi1*pi2**2*pi4 - lambda^6*lambda_1^7*pi1*pi2**2)/(pi1**2*pi4 - pi1**2 - pi1*pi2*pi4 + pi1*pi2 + pi2**2*pi4 - pi2**2)

tilde(34444) =
(-lambda^6*lambda^7*pi1**2*pi2*pi4 + lambda^6*lambda^7*pi1**2*pi4 - lambda^6*lambda^7*pi1**2 - lambda^6*lambda^7*pi1*pi2**2*pi4 - lambda^6*lambda^7*pi1*pi2*pi3 - 2*lambda^6*lambda^7*pi1*pi2*pi4 + 2*lambda^6*lambda^7*pi1*pi2 + lambda^6*lambda^7*pi2**2*pi4 - lambda^6*lambda^7*pi2**2 + lambda^6*lambda_1^7*pi1**2*pi2*pi4 - lambda^6*lambda_1^7*pi1**2*pi2 +

# Equalities k-states

In [ ]:
import sympy as sp
from functools import lru_cache

# ============================================================
# SETTINGS
# ============================================================
EXCLUDE_STATE_1 = True
LIGHT_SIMPLIFY = True

# ============================================================
# Helpers
# ============================================================
def parse_idx(s: str):
    s = s.strip().replace(" ", "")
    if not s or not all(ch.isdigit() for ch in s):
        raise ValueError("Index must be digits like 2244, 22444, 425544.")
    return tuple(int(ch) for ch in s)

def keep_idx(idx):
    return not (EXCLUDE_STATE_1 and (1 in idx))

# ============================================================
# MAIN
# ============================================================
def main():
    K = int(input("Enter kappa (number of states), e.g. 4: ").strip())
    if K < 2:
        raise ValueError("kappa must be >= 2")

    pi = sp.symbols(" ".join([f"pi{i}" for i in range(1, K + 1)]), nonzero=True)
    PI_SUM = sum(pi)

    def pisum(m):
        return sum(pi[:m])

    def simp_light(expr):
        if not LIGHT_SIMPLIFY:
            return expr
        expr = expr.subs({PI_SUM: 1})
        expr = sp.together(expr)
        expr = sp.cancel(expr)
        return expr

    @lru_cache(maxsize=None)
    def e_u(j, i):
        if i == 1:
            return sp.Integer(1)
        if j + i <= K + 1:
            return sp.Integer(1)
        if j == K + 2 - i:
            return -pisum(K + 1 - i) / pi[(K + 2 - i) - 1]
        return sp.Integer(0)

    @lru_cache(maxsize=None)
    def norm_u(i):
        if i == 1:
            return sp.Integer(1)
        a = K + 1 - i
        b = K + 2 - i
        denom = pi[(K + 2 - i) - 1]
        return simp_light(pisum(a) * pisum(b) / denom)

    @lru_cache(maxsize=None)
    def p0_bar(idx):
        denom = sp.Integer(1)
        for ir in idx:
            denom *= norm_u(ir)

        s = sp.Integer(0)
        for t in range(1, K + 1):
            prod_term = sp.Integer(1)
            for ir in idx:
                prod_term *= e_u(t, ir)
            s += pi[t - 1] * prod_term

        return simp_light(s / denom)

    def make_lam_vec(edge_label: str):
        lam1 = sp.Symbol(f"lambda_1^{edge_label}")
        lam  = sp.Symbol(f"lambda^{edge_label}")
        return [lam1] + [lam] * (K - 1)

    lam_vec6 = make_lam_vec("6")
    lam_vec7 = make_lam_vec("7")
    lam_vec8 = make_lam_vec("8")

    @lru_cache(maxsize=None)
    def barq4(idx4):
        i1, i2, i3, i4 = idx4
        s = sp.Integer(0)
        for y in range(1, K + 1):
            s += norm_u(y) * lam_vec6[y - 1] * p0_bar((i1, i2, y)) * p0_bar((y, i3, i4))
        return simp_light(s)

    @lru_cache(maxsize=None)
    def tildeq4(idx4):
        den = p0_bar(idx4)
        return sp.nan if den == 0 else simp_light(barq4(idx4) / den)

    @lru_cache(maxsize=None)
    def barq5(idx5):
        i1, i2, i3, i4, i5 = idx5
        s = sp.Integer(0)
        for x in range(1, K + 1):
            s += norm_u(x) * lam_vec7[x - 1] * barq4((i1, i2, i3, x)) * p0_bar((x, i4, i5))
        return simp_light(s)

    @lru_cache(maxsize=None)
    def tildeq5(idx5):
        den = p0_bar(idx5)
        return sp.nan if den == 0 else simp_light(barq5(idx5) / den)

    @lru_cache(maxsize=None)
    def barq6(idx6):
        i1, i2, i3, i4, i5, i6 = idx6
        s = sp.Integer(0)
        for x in range(1, K + 1):
            s += norm_u(x) * lam_vec8[x - 1] * barq5((i1, i2, i3, i4, x)) * p0_bar((x, i5, i6))
        return simp_light(s)

    @lru_cache(maxsize=None)
    def tildeq6(idx6):
        den = p0_bar(idx6)
        return sp.nan if den == 0 else simp_light(barq6(idx6) / den)

    def equal_under_simplex(expr):
        expr = simp_light(expr)
        expr = sp.together(expr)
        num, den = sp.fraction(expr)
        num = sp.expand(num)

        G = sp.groebner([PI_SUM - 1], *pi, order="lex", domain="EX")
        rem = G.reduce(num)[1]
        rem = sp.factor(rem)
        return (rem == 0), rem, den

    mode = input("Choose object (quartet / 5 / 6): ").strip().lower()
    if mode not in ["quartet", "5", "6"]:
        raise ValueError("Mode must be one of: quartet, 5, 6.")

    s1 = input("Enter index #1 (e.g. 2244 / 22444 / 425544): ").strip()
    s2 = input("Enter index #2: ").strip()

    idx1 = parse_idx(s1)
    idx2 = parse_idx(s2)

    if not keep_idx(idx1) or not keep_idx(idx2):
        raise ValueError("Index rejected: it contains state 1 while EXCLUDE_STATE_1=True.")

    def tilde_value(idx):
        if mode == "quartet":
            if len(idx) != 4:
                raise ValueError("quartet requires length-4 indices.")
            return tildeq4(idx)
        if mode == "5":
            if len(idx) != 5:
                raise ValueError("5 requires length-5 indices.")
            return tildeq5(idx)
        if mode == "6":
            if len(idx) != 6:
                raise ValueError("6 requires length-6 indices.")
            return tildeq6(idx)
        raise ValueError("Internal error.")

    v1 = tilde_value(idx1)
    v2 = tilde_value(idx2)
    diff = v1 - v2

    naive_is_zero = (simp_light(diff) == 0)
    is_zero, rem, den = equal_under_simplex(diff)

    print("\n--- Results ---")
    print(f"tilde({s1}) =\n{v1}")
    print(f"\ntilde({s2}) =\n{v2}")

    print("\n--- Equality check ---")
    print("Naive (after sum(pi)=1) ->", naive_is_zero)
    print("Under sum(pi)=1 (Groebner remainder) ->", is_zero)

    if not is_zero:
        print("\nRemainder of numerator mod <sum(pi)-1>:")
        print(rem)
        print("\nDenominator (for reference):")
        print(den)

if __name__ == "__main__":
    main()


Enter kappa (number of states), e.g. 4: 5
Choose object (quartet / 5 / 6): 6
Enter index #1 (e.g. 2244 / 22444 / 425544): 442355
Enter index #2: 553244

--- Results ---
tilde(442355) =
(-lambda^6*lambda^7*lambda^8*pi1**4 - 4*lambda^6*lambda^7*lambda^8*pi1**3*pi2 - 3*lambda^6*lambda^7*lambda^8*pi1**3*pi3 - 2*lambda^6*lambda^7*lambda^8*pi1**3*pi4 - 6*lambda^6*lambda^7*lambda^8*pi1**2*pi2**2 - 9*lambda^6*lambda^7*lambda^8*pi1**2*pi2*pi3 - 6*lambda^6*lambda^7*lambda^8*pi1**2*pi2*pi4 - 3*lambda^6*lambda^7*lambda^8*pi1**2*pi3**2 - 4*lambda^6*lambda^7*lambda^8*pi1**2*pi3*pi4 - lambda^6*lambda^7*lambda^8*pi1**2*pi4**2 + lambda^6*lambda^7*lambda^8*pi1**2*pi5**2 + lambda^6*lambda^7*lambda^8*pi1**2*pi5 - 2*lambda^6*lambda^7*lambda^8*pi1**2 - 4*lambda^6*lambda^7*lambda^8*pi1*pi2**3 - 9*lambda^6*lambda^7*lambda^8*pi1*pi2**2*pi3 - 6*lambda^6*lambda^7*lambda^8*pi1*pi2**2*pi4 - 6*lambda^6*lambda^7*lambda^8*pi1*pi2*pi3**2 - 8*lambda^6*lambda^7*lambda^8*pi1*pi2*pi3*pi4 - 2*lambda^6*lambda^7*lambda^8*pi1